<a href="https://colab.research.google.com/github/codingshinobi/Deloitte_Machinehack_Loan_Default_prediction/blob/main/loan_defaulter_hyperas_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install hyperas
! pip install hyperopt

In [ ]:
!pip install tensorflow

In [ ]:
# Standard Imports
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import log_loss

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Input, Dense, Dropout, Activation

from hyperopt import tpe, STATUS_OK, Trials
from hyperas import optim
from hyperas.distributions import choice, uniform

## Let's create a function which processes the data in such a way that that the data is directly fed into the model

In [ ]:
def data():  
  global test
  # Importing data
  train = pd.read_csv(r"D:\DATASETS\DELOITTE\Participants_Data_PLD\train.csv")
  test = pd.read_csv(r"D:\DATASETS\DELOITTE\Participants_Data_PLD\test.csv")

  # Dropping unnecessary columns in both test and training sets
  train = train.drop("ID", axis=1)
  test = test.drop(["Loan Status", "ID"], axis=1)

  # Let's split the data into features and target variables
  X = train.drop("Loan Status", axis=1)
  y = train["Loan Status"]

  # Splitting the data into training and validation sets
  X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

  # Converting non- numerical columns into numerica Columns
  for label, content in X_train.items():
    if not pd.api.types.is_numeric_dtype(content):
      encoder = LabelEncoder()
      X_train[label] = encoder.fit_transform(X_train[label])
      X_valid[label] = encoder.transform(X_valid[label])
      test[label] = encoder.transform(test[label])

  # Now we scale the data
  scaler = StandardScaler()
  X_train = scaler.fit_transform(X_train)
  X_valid = scaler.transform(X_valid)
  test = scaler.transform(test)

  return X_train, y_train, X_valid, y_valid

# Calling our function
X_train, y_train, X_valid, y_valid = data()

<ipython-input-8-fd6fe00dc778>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[label] = encoder.fit_transform(X_train[label])
<ipython-input-8-fd6fe00dc778>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_valid[label] = encoder.transform(X_valid[label])


## Let's create our model

In [ ]:
def model():
  # Set random seed
  tf.random.set_seed(42)

  # Create a model
  model = Sequential()
  model.add(Input(shape=X_train[0].shape))
  model.add(Dense({{choice([20, 30, 40])}}))
  model.add(Activation("relu"))
  model.add(Dropout({{uniform(0, 1)}}))
  model.add(Dense({{choice([10, 20, 30])}}))
  model.add(Activation("relu"))
  model.add(Dropout({{uniform(0, 1)}}))

  # Let's add another layer according to choice
  if {{choice(["a", "b"])}} == "b":
    model.add(Dense(5, activation="relu"))
    model.add(Dropout({{uniform(0, 1)}}))

  # Let's add the final layer
  model.add(Dense(1))
  model.add(Activation({{choice(["sigmoid", "softmax"])}}))

  # let's compile the model
  model.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(learning_rate={{choice([0.1, 0.001, 0.01, 3e-4])}}),
                metrics=["accuracy"])
  
  # Fit the model
  model.fit(X_train, y_train,
            validation_data=(X_valid, y_valid),
            epochs=100,
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)], verbose=0)
  
  # Evaluate our model
  loss, acc = model.evaluate(X_valid, y_valid, verbose=0)
  print("Validation accuracy :", acc)

  return {"loss" : -acc, "model" : model, 'status' : STATUS_OK}

In [ ]:
# Now let's optimize our model
best_run, best_model = optim.minimize(data=data,
                                      model=model,
                                      max_evals=10,
                                      algo=tpe.suggest,
                                      trials=Trials())

In [ ]:
log_loss(y_valid, best_model.predict(X_valid))

NameError: ignored

In [ ]:
best_run

In [ ]:
# Final model

# Create a model
final = Sequential([
  Input(shape=X_train[0].shape),
  Dense(100, activation="relu"),
  Dropout(0.6108763092812357),
  Dense(50, activation="relu"),
  Dropout(0.7371698374615214),
  Dense(25, activation="relu"),
  Dropout(0.21280043312755825),
  Dense(1, activation="sigmoid")
])

# Compile the model
final.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
              metrics=["accuracy"])

#Fit the model
history =   final.fit(X_train, y_train, epochs=100,
            validation_data=(X_valid, y_valid),
            callbacks=[tf.keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
log_loss(y_valid, final.predict(X_valid))

0.303732815514517

In [ ]:
predictions=final.predict(test).flatten()
submissions = pd.DataFrame()
predictions

array([0.10211161, 0.09748694, 0.10211161, ..., 0.10009122, 0.10211161,
       0.10188353], dtype=float32)

In [ ]:
submissions["Loan Status"] = predictions
submissions.to_csv("predictions.csv", index=False)